In [21]:
pip freeze > ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [17]:
import zipfile
import pandas as pd
from surprise import SVD, Reader, Dataset
from surprise.model_selection import cross_validate

In [15]:

path = r"P:\Python\GitHub\movie_recommendation_system\data\raw\archive.zip"
csv_file = "ratings_small.csv"


with zipfile.ZipFile(path,"r") as z:
# Ensure that the CSV file exists within the zip
    # Open the CSV file within the zip file
    print(z.namelist())
    with z.open(csv_file) as f:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(f)
        df.columns = df.columns.str.lower()
df.head()

['credits.csv', 'keywords.csv', 'links.csv', 'links_small.csv', 'movies_metadata.csv', 'ratings.csv', 'ratings_small.csv']


,userid,movieid,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [20]:
df.head(10)

,userid,movieid,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [22]:
def train_model(data):
    # Define a reader object by specifying the rating scale
    reader = Reader(rating_scale=(1, 5))
    
    # Load the dataset from DataFrame, Surprise requires dataset in specific format
    data = Dataset.load_from_df(data[['userid', 'movieid', 'rating']], reader)
    
    # Build the training set from the data
    trainset = data.build_full_trainset()
    
    # Use SVD algorithm for matrix factorization
    algo = SVD()

    # Train the algorithm on the training set
    algo.fit(trainset)
    return algo

def evaluate_model(algo, data):
    # Run 5-fold cross-validation and print results
    cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [23]:

model = train_model(df)

evaluate_model(model, Dataset.load_from_df(df[['userid', 'movieid', 'rating']], Reader(rating_scale=(1, 5))))

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9001  0.8976  0.8986  0.8949  0.8937  0.8970  0.0023  
MAE (testset)     0.6942  0.6895  0.6914  0.6895  0.6885  0.6906  0.0020  
Fit time          1.42    1.41    1.64    1.41    1.52    1.48    0.09    
Test time         0.21    0.27    0.17    0.15    0.34    0.23    0.07    


In [27]:
Dataset.load_from_df(df[['userid', 'movieid', 'rating']], Reader(rating_scale=(1, 5)))